In [3]:
from dotenv import load_dotenv
import os
load_dotenv()

python-dotenv could not parse statement starting at line 1
python-dotenv could not parse statement starting at line 7


True

In [4]:
os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY")

In [5]:
import pandas as pd
df = pd.read_csv('Data.csv')

In [7]:
df.head()

,Product,Review
0,Queen Size Sheet Set,I ordered a king size set. My only criticism w...
1,Waterproof Phone Pouch,"I loved the waterproof sac, although the openi..."
2,Luxury Air Mattress,This mattress had a small hole in the top of i...
3,Pillows Insert,This is the best throw pillow fillers on Amazo...
4,Milk Frother Handheld\r\n,I loved this product. But they only seem to l...


Simple LLM chain

In [8]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.chains import LLMChain

In [9]:
llm = ChatOpenAI(temperature=0.9)

/var/folders/s8/qyjb36g92fs3ztdqk120mmkw0000gn/T/ipykernel_15966/3549645701.py:1: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import ChatOpenAI``.
  llm = ChatOpenAI(temperature=0.9)


In [10]:
prompt = ChatPromptTemplate.from_template(
    "What is the best name to describe a company that makes {product} ?"
)

In [11]:
chain = LLMChain(
    llm = llm,
    prompt = prompt
)

/var/folders/s8/qyjb36g92fs3ztdqk120mmkw0000gn/T/ipykernel_15966/4255761920.py:1: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use :meth:`~RunnableSequence, e.g., `prompt | llm`` instead.
  chain = LLMChain(


In [12]:
product = "Queen size sheet set"
chain.run(product)

/var/folders/s8/qyjb36g92fs3ztdqk120mmkw0000gn/T/ipykernel_15966/12625076.py:2: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  chain.run(product)


'"Regal Linens"'

Sequentail chain : - The idea is to combine multiple chains where the output of the one chain is the input of the next chain.

Types: 

1. Simple Sequential Chain : - Single Input/Output

2. Sequential Chain : - Multiple Inputs/Outputs

Simple sequential chain

In [13]:
from langchain.chains import SimpleSequentialChain

In [14]:
llm = ChatOpenAI(temperature = 0.9)

first_prompt = ChatPromptTemplate.from_template(
    "What is the best name to describe a company that makes {product}?"
)

In [ ]:
#chain 1
chain_one = LLMChain(
    llm=llm,
    prompt = first_prompt
)

In [16]:
second_prompt = ChatPromptTemplate.from_template(
    "rite a 20 words description for the following company {company_name}"
)

In [18]:
#Chain 2
chain_two = LLMChain(
    llm =llm,
    prompt = second_prompt
)

In [19]:
simple_sequential_chain = SimpleSequentialChain(chains = [chain_one,chain_two],verbose=True)

In [20]:
simple_sequential_chain.run(product)



> Entering new SimpleSequentialChain chain...
Regal Linens
Regal Linens offers luxurious bedding and home decor items that add a touch of elegance and sophistication to any space.

> Finished chain.


'Regal Linens offers luxurious bedding and home decor items that add a touch of elegance and sophistication to any space.'

Regular Sequential chain:- It has multiple inputs and multiple outputs

In [21]:
from langchain.chains import SequentialChain

In [23]:
llm = ChatOpenAI(temperature = 0.9)

# First prompt template
first_prompt = ChatPromptTemplate.from_template(
    "Translate the following review into english:"
    "\n\n{Review}"
)

chain_one = LLMChain(
    llm = llm,
    prompt = first_prompt,
    output_key = "English_Review"
)

In [24]:
# Second prompt template convert to english

second_prompt = ChatPromptTemplate.from_template(
    "Can you summarize the following review in sentence 1"
    "\n\n{English_Review}"
)

chain_second = LLMChain(
    llm = llm,
    prompt = second_prompt,
    output_key = "summary"
)


In [25]:
#Third prompt template

third_prompt = ChatPromptTemplate.from_template(
    "What language is the following review: \n\n{Review}"
)
chain_three = LLMChain(
    llm = llm,
    prompt = third_prompt,
    output_key = "language"
)

In [26]:
# Forth prompt Template

forth_prompt = ChatPromptTemplate.from_template(
    "Write a follow up response to the following"
    "Summary in the specified language:"
    "\n\n Summary: {summary} \n\n Language: {language}"
)

chain_four = LLMChain(
    llm =llm,
    prompt = forth_prompt,
    output_key="followup_message"
)

In [29]:
overall_chain = SequentialChain(
    chains = [chain_one,chain_second,chain_three,chain_four],
    input_variables = ["Review"],
    output_variables = ["English_Review","summary","followup_message"],
    verbose = True
)

In [30]:
review = df.Review[5]
overall_chain(review)

/var/folders/s8/qyjb36g92fs3ztdqk120mmkw0000gn/T/ipykernel_15966/1992003631.py:2: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  overall_chain(review)




> Entering new SequentialChain chain...

> Finished chain.


{'Review': "Je trouve le goût médiocre. La mousse ne tient pas, c'est bizarre. J'achète les mêmes dans le commerce et le goût est bien meilleur...\r\nVieux lot ou contrefaçon !?",
 'English_Review': "I find the taste mediocre. The foam does not hold, it's weird. I buy the same ones in stores and the taste is much better... Old batch or counterfeit!?",
 'summary': 'The reviewer found the taste of the product mediocre and suspected it may be an old batch or counterfeit.',
 'followup_message': "Je vous remercie pour votre avis. Nous sommes désolés d'apprendre que vous avez trouvé le goût du produit médiocre. Nous prenons la qualité de nos produits très au sérieux et nous allons vérifier s'il s'agit d'un lot ancien ou contrefait. Votre satisfaction est notre priorité et nous ferons tout notre possible pour rectifier la situation. Merci de nous avoir informés."}

Router Chain:- Route the input to the chain depending upon what exactly the output is.

In [31]:
physics_template = """You are a very smart physics professor. \
You are great at answering questions about physics in a concise\
and easy to understand manner. \
When you don't know the answer to a question you admit\
that you don't know.

Here is a question:
{input}"""


math_template = """You are a very good mathematician. \
You are great at answering math questions. \
You are so good because you are able to break down \
hard problems into their component parts,
answer the component parts, and then put them together\
to answer the broader question.

Here is a question:
{input}"""

history_template = """You are a very good historian. \
You have an excellent knowledge of and understanding of people,\
events and contexts from a range of historical periods. \
You have the ability to think, reflect, debate, discuss and \
evaluate the past. You have a respect for historical evidence\
and the ability to make use of it to support your explanations \
and judgements.

Here is a question:
{input}"""


computerscience_template = """ You are a successful computer scientist.\
You have a passion for creativity, collaboration,\
forward-thinking, confidence, strong problem-solving capabilities,\
understanding of theories and algorithms, and excellent communication \
skills. You are great at answering coding questions. \
You are so good because you know how to solve a problem by \
describing the solution in imperative steps \
that a machine can easily interpret and you know how to \
choose a solution that has a good balance between \
time complexity and space complexity.

Here is a question:
{input}"""

In [32]:
prompt_infos = [
    {
        "name": "physics",
        "description": "Good for answering questions about physics",
        "prompt_template": physics_template
    },
    {
        "name": "math",
        "description": "Good for answering questions about maths",
        "prompt_template": math_template
    },
    {
        "name": "History",
        "description": "Good for answering questions about history",
        "prompt_template": history_template
    },
    {
        "name": "Computer Science",
        "description": "Good for answering questions about computer science",
        "prompt_template": computerscience_template
    }
]

In [36]:
from langchain.chains.router import MultiPromptChain
from langchain.chains.router.llm_router import LLMRouterChain, RouterOutputParser
from langchain.prompts import PromptTemplate

In [37]:
llm=ChatOpenAI(temperature = 0.9)

In [43]:
destination_chains = {}
for p_info in prompt_infos:
    name = p_info["name"]
    prompt_template = p_info["prompt_template"]
    prompt = ChatPromptTemplate.from_template(
             template = prompt_template
    )
    chain = LLMChain(
        llm = llm,
        prompt = prompt
    )
    destination_chains[name] = chain
destinations = [f"{p['name']}: {p['description']}" for p in prompt_infos]
destinations_str = "\n".join(destinations)

In [55]:
print(destinations_str)

physics: Good for answering questions about physics
math: Good for answering questions about maths
History: Good for answering questions about history
Computer Science: Good for answering questions about computer science


In [44]:
default_prompt = ChatPromptTemplate.from_template("{input}")
default_chain = LLMChain(
    llm = llm,
    prompt = default_prompt
)

In [45]:
MULTI_PROMPT_ROUTER_TEMPLATE = """Given a raw text input to a \
language model select the model prompt best suited for the input. \
You will be given the names of the available prompts and a \
description of what the prompt is best suited for. \
You may also revise the original input if you think that revising\
it will ultimately lead to a better response from the language model.

<< FORMATTING >>
Return a markdown code snippet with a JSON object formatted to look like:
```json
{{{{
    "destination": string \ "DEFAULT" or name of the prompt to use in {destinations}
    "next_inputs": string \ a potentially modified version of the original input
}}}}
```

REMEMBER: The value of “destination” MUST match one of \
the candidate prompts listed below.\
If “destination” does not fit any of the specified prompts, set it to “DEFAULT.”
REMEMBER: "next_inputs" can just be the original input \
if you don't think any modifications are needed.

<< CANDIDATE PROMPTS >>
{destinations}

<< INPUT >>
{{input}}

<< OUTPUT (remember to include the ```json)>>"""

In [46]:
router_template = MULTI_PROMPT_ROUTER_TEMPLATE.format(
    destinations=destinations_str
)

router_prompt = PromptTemplate(
    template=router_template,
    input_variables=["input"],
    output_parser=RouterOutputParser(),
)

router_chain = LLMRouterChain.from_llm(
    llm,
    router_prompt
)

In [47]:
chain = MultiPromptChain(
    router_chain= router_chain,
    destination_chains=destination_chains,
    default_chain=default_chain,
    verbose=True
)

/var/folders/s8/qyjb36g92fs3ztdqk120mmkw0000gn/T/ipykernel_15966/530885520.py:1: LangChainDeprecationWarning: Please see migration guide here for recommended implementation: https://python.langchain.com/docs/versions/migrating_chains/multi_prompt_chain/
  chain = MultiPromptChain(


In [56]:
chain.run("What is x86 architecture")



> Entering new MultiPromptChain chain...
Computer Science: {'input': 'What is x86 architecture'}
> Finished chain.


'x86 architecture refers to a family of instruction set architectures based on the Intel 8086 microprocessor. It has evolved over the years and is commonly used in most personal computers and servers today. The x86 architecture includes a set of instructions that the processor can execute, as well as memory management and input/output capabilities. This architecture is what allows software developers to write programs that can run on x86-based systems.'